In [1]:
!pip install import-ipynb

In [2]:
#%cd /content/drive/MyDrive/NLP



# TensorBoard를 사용하면 손실 및 정확도와 같은 측정 항목을 추적 및 시각화하는 것, 모델 그래프를 시각화하는 것, 히스토그램을 보는 것, 이미지를 출력하는 것 등이 가능
import import_ipynb
import torch.utils.tensorboard as tensorboard 
import numpy as np

import os

from vision_model_final import Mini_Xception,NN,VGG16
from vision_dataset_final import create_train_dataloader, create_val_dataloader

import torch
import torch.nn as nn
import torch.optim

from sklearn.metrics import accuracy_score
import random

importing Jupyter notebook from vision_model_final.ipynb
importing Jupyter notebook from vision_dataset_final.ipynb


C:\Users\user\anaconda3\envs\ml\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
torch.cuda.empty_cache()

**잠깐 ✔ 랜덤시드 고정이란 무엇인가?**
> 학습된 모델의 결과를 동일하게 재현(Reproduction)하는 것은 여러가지 상황에서 팔요합니다.  
> 모델을 돌릴 때마다 결과가 달라지지 않도록 고정하는 것입니다.

- 수상자가 되어 코드의 정합성을 검증 받게 될 경우,

- 경진대회 참가 도중 팀을 이루어 결과를 공유해야 되는 경우,

- 논문을 작성하여 그 결과를 Reproduction 해야하는 경우 등 여러 상황에서 필요합니다.

참고 자료:
https://dacon.io/codeshare/2363
https://pytorch.org/docs/stable/notes/randomness.html

In [4]:
seed_val = 2022
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [5]:
class FocalLoss(nn.Module):
    """Implementation of Facal Loss"""
    def __init__(self, weight=None, gamma=2, reduction="mean"):
        super(FocalLoss, self).__init__()
        self.weighted_cs = nn.CrossEntropyLoss(weight=weight, reduction="none")
        self.cs = nn.CrossEntropyLoss(reduction="none")
        self.gamma = gamma
        self.reduction = reduction
        
    def forward(self, predicted, target):
        """
        predicted: [batch_size, n_classes]
        target: [batch_size]
        """
        pt = 1/torch.exp(self.cs(predicted,target))
        #shape: [batch_size]
        entropy_loss = self.weighted_cs(predicted, target)
        #shape: [batch_size]
        focal_loss = ((1-pt)**self.gamma)*entropy_loss
        #shape: [batch_size]
        if self.reduction =="none":
            return focal_loss
        elif self.reduction == "mean":
            return focal_loss.mean()
        else:
            return focal_loss.sum()

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# tensorboard 만들기(epoch마다 loss와 정확도 변화의 경향을 파악하는 데 좋다.) -> 선택 사항
writer = tensorboard.SummaryWriter('checkpoint/model_weights/')
# 모델 정의하기
## 여기에 코드 작성
# 모델 학습 파라미터 지정 # todo code
learning_rate = 1e-2 # 무조건 작다고 좋지 않다.
epochs = 50

model = VGG16().to(device)  # 모델 instantiation for training
# 옵티마이저 정의
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate
                            ,nesterov=True,momentum=0.9,weight_decay=1e-6)


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', 
                                            factor=0.75, patience=5)

class_weights=torch.FloatTensor([0.8608,0.9848,0.8572,
                                 0.7486,0.8270,0.8317,0.8895]).to(device)
# 손실 함수 정의
criterion = FocalLoss(class_weights)
model.zero_grad() # 그래디언트 초기화
PATH = "./checkpoint/model_weights/weights_epoch_"
def main(): # 코드가 실행될 main 함수를 만들어봅시다.
    

    train_dataloader =create_train_dataloader(batch_size=16)
    val_dataloader = create_val_dataloader(batch_size=16)

    print("Training!")
    
    
    for epoch in range(0, epochs):
        train_loss = train(model,   train_dataloader,epoch)
        val_loss, accuracy = validation(model,  val_dataloader, epoch)
        scheduler.step(val_loss)


        
        final_loss, accuracy=validation(model,val_dataloader,epoch)
        # 매 epoch 마다 checkpoint로 model을 저장할 필요가 있습니다.
        # https://tutorials.pytorch.kr/recipes/recipes/saving_and_loading_a_general_checkpoint.html
        ## 여기에 코드 작성
        writer.add_scalar('train_loss',train_loss, epoch)
        writer.add_scalar('val_loss',val_loss, epoch)
        writer.add_scalar('accuracy',accuracy, epoch)

        checkpoint_state = {
                'mini_xception': model.state_dict(),
                "epoch": epoch
            }
        torch.save(checkpoint_state, PATH+str(epoch+1)+'.pth.tar')


    
    

    writer.close() # Tensorboard 닫기


# epoch 마다 train하는 코드 -> 정확도를 얻는 코드(중간미션에서 with torch.no_grad()로 작성하였던 부분)를 작성해야 합니다.
# train_loss을 얻는 함수와 정확도를 얻는 함수를 각각 작성하는 것이 편리합니다.
# loss = criterion(input,label)로 손실을 구할 수 있습니다. 이때 lnput과 label의 shape을 고려해야 합니다.
# input = (minibatch,7) -> 감정이 7개의 class이므로 / label = (minibatch, )의 shape을 가집니다.
# ***따라서 criterion을 이용해 loss을 구하기 전 input의 shape을 (minibatch,7)이 아니라면 꼭 이 shape으로 바꿔주어야 합니다! -> squeeze 함수 시용해보기
# 알면 좋은 정보: from sklearn.metrics import accuracy_score을 통해 accuracy_score(label,prediction)로 정확도를 얻을 수 있습니다.

def train(model, dataloader, epoch): # train 과정에서의 손실을 계산하는 함수 작성
    model.train()
    model.to(device)
    ## 여기에 코드 작성
    # 더 필요한 변수나 작업이 있다면 작성해봅시다. 정해진 틀은 없으며 자유롭게 작성하시면 됩니다. loss나 optimizer로 어떤 것을 사용할건지 등..
    
    losses = []
    total_loss=0
    total=0
    print("")
    print('========{:}번째 Epoch / 전체 {:}회 ========'.format(epoch + 1, epochs))
    print('훈련 중')
    
    for images, labels in dataloader:
        ## 여기에 코드 작성
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        losses.append(loss.cpu().item())
        loss.backward()
        
        optimizer.step()
        
        
    
        ## 여기에 코드 작성
        # back propagation을 떠올려보며 train 코드를 마무리하기
    # 평균 로스 계산
    
    print(f'training @ epoch {epoch+1} .. loss = {round(loss.item(),4)}')
    
    #### 검증 ####
    
    print("")
    print("검증 중")

    return round(np.mean(losses).item(),4)

def validation(model, dataloader, epoch): 
  # 중간미션 1에서 with torch.no_grad(): 부분에 작성하였던 test accuracy를 구하는 방법을 떠올려보면 됩니다. 
  # train 과정에서의 손실도 중요하나 최종적으로 random data을 넣어서 test를 한 후의 정확도를 얻어야 합니다.
    model.eval() # 왜 설정할까요?
    model.to(device)
    ## 여기에 코드 작성
    # 변수 필요한거 설정 (loss로 어떤걸 사용할건지 등)
    losses = []

    total_pred = []
    total_labels = []

    with torch.no_grad(): 
        # 학습이 x 모델 성능 평가 따라서 autograd하지 않음!!
        correct = 0 # 맞은 개수 세기 위해 int 정의
        total = 0 # 전체 실행 개수를 세서 정확도를 계산하기 위해 int 정의
        for images, labels in dataloader:
            minibatch = images.shape[0]
            images = images.to(device)
            labels = labels.to(device)
                    
            # 이제 images을 모델에 통과시켜 얻은 예측값으로 loss을 구해야 합니다.
            # 이를 criterion 함수에 넣기 위해선 먼저 예측값을 (minibatch, # of class)의 형태로 shape을 맞춰주어야 합니다. minibatch의 수를 알 수 있다면 편합니다!
            ## 여기에 코드 작성
            
            outputs = model(images)
            outputs = outputs.reshape(minibatch, -1)
            loss = criterion(outputs, labels)
            losses.append(loss.cpu().item())
            
            
            # 이제 model을 거쳐 얻은 텐서에서 가장 큰 값을 갖는 곳의 index을 알아야 emotion이 무엇인지 알 수 있습니다.
            # index을 구해보고 이를 통해 index와 label간 차이로 정확도를 구합시다. 참고: https://pytorch.org/docs/stable/generated/torch.max.html
            # Scikit learn의 accuracy_score을 이용하고 싶다면 어떤식으로 예측값과 정답을 처리해야 할 지 고민해 봅시다.
            ## 여기에 코드 작성
            _, indexes = torch.max(outputs, axis=1)
            # print(indexes.shape, labels.shape)
            total_pred.extend(indexes.cpu().detach().numpy())
            total_labels.extend(labels.cpu().detach().numpy())
            
            
        final_loss = np.mean(losses).item()
        accuracy = accuracy_score(total_labels, total_pred)

        
        # 최종 loss와 정확도(소수점 넷째 자리까지)를 출력해봅시다. 
        print(f'Val loss = {final_loss} .. Accuracy = {accuracy:.4f} ')

        return final_loss, accuracy

if __name__ == "__main__": # 앞서 작성한 main 함수 실행
    main()


Training!

========1번째 Epoch / 전체 50회 ========
훈련 중
training @ epoch 1 .. loss = 1.2089

검증 중
Val loss = 1.0273858655823602 .. Accuracy = 0.2903 
Val loss = 1.027963744799296 .. Accuracy = 0.2926 

========2번째 Epoch / 전체 50회 ========
훈련 중
training @ epoch 2 .. loss = 0.812

검증 중
Val loss = 0.9381595322820875 .. Accuracy = 0.3608 
Val loss = 0.9383374778429667 .. Accuracy = 0.3594 

========3번째 Epoch / 전체 50회 ========
훈련 중
training @ epoch 3 .. loss = 1.2355

검증 중
Val loss = 0.8951509116755592 .. Accuracy = 0.3706 
Val loss = 0.8950870335102081 .. Accuracy = 0.3742 

========4번째 Epoch / 전체 50회 ========
훈련 중
training @ epoch 4 .. loss = 0.5403

검증 중
Val loss = 0.8519985598988004 .. Accuracy = 0.4230 
Val loss = 0.8499839372105069 .. Accuracy = 0.4266 

========5번째 Epoch / 전체 50회 ========
훈련 중
training @ epoch 5 .. loss = 0.4971

검증 중
Val loss = 0.7298269497023688 .. Accuracy = 0.4937 
Val loss = 0.7259974286291334 .. Accuracy = 0.4923 

========6번째 Epoch / 전체 50회 ========
훈련 중
training @

training @ epoch 44 .. loss = 0.3821

검증 중
Val loss = 0.4429959622356627 .. Accuracy = 0.6718 
Val loss = 0.44238551417986555 .. Accuracy = 0.6712 

========45번째 Epoch / 전체 50회 ========
훈련 중
training @ epoch 45 .. loss = 0.1806

검증 중
Val loss = 0.46524938242303 .. Accuracy = 0.6486 
Val loss = 0.4545478990342882 .. Accuracy = 0.6523 

========46번째 Epoch / 전체 50회 ========
훈련 중
training @ epoch 46 .. loss = 0.1295

검증 중
Val loss = 0.440326739748319 .. Accuracy = 0.6643 
Val loss = 0.43747604300578435 .. Accuracy = 0.6590 

========47번째 Epoch / 전체 50회 ========
훈련 중
training @ epoch 47 .. loss = 0.2075

검증 중
Val loss = 0.4989240498012967 .. Accuracy = 0.6247 
Val loss = 0.4999567151069641 .. Accuracy = 0.6205 

========48번째 Epoch / 전체 50회 ========
훈련 중
training @ epoch 48 .. loss = 0.6712

검증 중
Val loss = 0.45153025441699557 .. Accuracy = 0.6470 
Val loss = 0.4467769331402249 .. Accuracy = 0.6489 

========49번째 Epoch / 전체 50회 ========
훈련 중
training @ epoch 49 .. loss = 0.3371

검증 중
Val los

In [7]:
# 저장해놓은 텐서보드 로드하기(in colab) - loss와 accuracy의 경향 확인
%load_ext tensorboard
%tensorboard --logdir ./checkpoint/model_weights

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
C:\Users\user\anaconda3\envs\ml\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\ml\lib\runpy.py", line 194, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\user\anaconda3\envs\ml\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\user\anaconda3\envs\ml\Scripts\tensorboard.exe\__main__.py", line 7, in <module>
  File "C:\Users\user\anaconda3\envs\ml\lib\site-packages\tensorboard\main.py", line 46, in run_main
    app.run(tensorboard.main, flags_parser=tensorboard.configure)
  File "C:\Users\user\anaconda3\envs\ml\lib\site-packages\absl\app.py", line 308, in run
    _run_main(main, args)
  File "C:\U